In [0]:
from delta.tables import DeltaTable
# Load Delta table from path
delta_path = "/Volumes/workspace/ecommerce/ecommerce_data/delta/events"
deltaTable = DeltaTable.forPath(spark, delta_path)


In [0]:
# Read updates
updates = (
    spark.read
    .csv(
        "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
        header=True,
        inferSchema=True
    )
    .dropDuplicates(["user_session", "event_time"])
)

In [0]:
# MERGE for incremental updates
(
    deltaTable.alias("t")
    .merge(
        updates.alias("s"),
        "t.user_session = s.user_session AND t.event_time = s.event_time"
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Time travel
v0 = spark.read.format("delta").option("versionAsOf", 0).load("/Volumes/workspace/ecommerce/ecommerce_data/delta/events")
yesterday = spark.read.format("delta") \
    .option("timestampAsOf", "2024-01-01").load("/Volumes/workspace/ecommerce/ecommerce_data/delta/events")



In [0]:
# Optimize
spark.sql("OPTIMIZE events_table ZORDER BY (event_type, user_id)")
spark.sql("VACUUM events_table RETAIN 168 HOURS")

DataFrame[path: string]